In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc



CPU times: user 573 ms, sys: 2.17 s, total: 2.74 s
Wall time: 398 ms


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

test_o = test.copy()

(69713, 22)
(30037, 21)
CPU times: user 13 s, sys: 6.78 s, total: 19.7 s
Wall time: 19.7 s


In [6]:
df.shape

(69713, 37416)

In [5]:
%%time

rid = []

for i in df:
    if df[i].value_counts().head(1).values[0]>df.shape[0]*0.90:
        rid.append(i)
print len(rid), df.shape[1]-len(rid)

37380 36
CPU times: user 34.2 s, sys: 3 ms, total: 34.2 s
Wall time: 34.2 s


### from 186 features

In [5]:
from keep import keep
record = []

dic = {}
for i in keep:
    dic[i]=df[i].value_counts().head(1).values[0]

pic = pd.Series(dic).sort_values(ascending=True)
rm = list(pic.index)    
len(rm)

186

In [6]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}



In [8]:
def search(i):
    global record
    
    update_rid = set(df.columns) - set(rm[:i])
    print len(df.columns) - len(update_rid)


    train = df.drop(update_rid, axis=1)
    test = test_o.drop(update_rid, axis=1)

    s = StandardScaler()
    
    train = s.fit_transform(train)
    test = s.transform(test)

    print train.shape, test.shape    
    
    m = lambda *x: [i.shape for i in x]
    X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
    print m(X_train, y_train, X_test, y_test)

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)    
    
    watchlist = [(d_train, 'train'), (d_test, 'test')]

    
    
    start=time()

    clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                    verbose_eval=200,  maximize=True,
                   )



    print '!!! used %.2f s'%(time()-start)    
    record.append([i, clf.best_iteration, clf.best_score])
    print record[-1]

In [10]:
# for i in range(130, 187):
#     search(i)
# len(record)

In [11]:
rs = pd.DataFrame(record, columns=['feature','round','score'])
# rs
rs.to_csv('search2.csv',index=False)
rs[rs['score']>.859].sort_values(by=['score'],ascending=False)

,feature,round,score
55,185,325,0.861806
26,156,292,0.861340
11,141,280,0.861160
21,151,284,0.860855
27,157,329,0.860502
45,175,297,0.860473
30,160,303,0.860281
44,174,288,0.860161
7,137,306,0.859951
3,133,330,0.859866


In [86]:
update_rid = set(df.columns) - set(rm[:100])
len(df.columns) - len(update_rid)

100

In [87]:
%%time

test = test_o.copy()
train = df.drop(update_rid, axis=1)
test = test.drop(update_rid, axis=1)

s = StandardScaler()
train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape

(69713, 100) (30037, 100)
CPU times: user 1.08 s, sys: 1.31 s, total: 2.4 s
Wall time: 2.4 s


### from here

In [88]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
print m(X_train, y_train, X_test, y_test)

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

[(55770, 100), (55770,), (13943, 100), (13943,)]
CPU times: user 151 ms, sys: 28 ms, total: 179 ms
Wall time: 178 ms


In [90]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.746156	test-auc:0.75242
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[50]	train-auc:0.873785	test-auc:0.839442
[100]	train-auc:0.917732	test-auc:0.845188
[150]	train-auc:0.953892	test-auc:0.851299
[200]	train-auc:0.973941	test-auc:0.855542
[250]	train-auc:0.985383	test-auc:0.856642
[300]	train-auc:0.991301	test-auc:0.856767
[350]	train-auc:0.994571	test-auc:0.857295
[400]	train-auc:0.996707	test-auc:0.855118
[450]	train-auc:0.997917	test-auc:0.853925
Stopping. Best iteration:
[272]	train-auc:0.988468	test-auc:0.857827



'!!! used 13.79 s'

In [91]:
clf.best_score

0.857827

In [27]:
%%time
dtest = xgb.DMatrix(test)
predb = clf.predict(dtest)

submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

CPU times: user 1.32 s, sys: 7 ms, total: 1.33 s
Wall time: 102 ms


___

In [28]:
submit.to_csv('./submit/xgb_rm_rd.csv',index=False)

#### lb .852388

In [60]:
# clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba